In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os 

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.environ["GEMINI_API_KEY"])

## Tools

In [2]:
from langchain_core.tools import StructuredTool
import numpy as np


def sum_random_number(a: int) -> int:
    """Sum a random number the input number.

    Parameters
    ----------
    a : int
        First number to multiply

    Returns
    -------
    int
        The sum of the two numbers
    """
    random_number = 3
    return a + random_number


calculator = StructuredTool.from_function(func=sum_random_number, name="calculator")

## Agent

Simple System Prompt

In [3]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

from ReAct_system_prompt import react_system_prompt
special_instructions = """You must answer in Spanish."""
system_prompt = react_system_prompt.replace("$/special_instructions/$", special_instructions)
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")
memory = MemorySaver()
langgraph_agent_executor = create_react_agent(llm, [calculator], prompt=special_instructions, checkpointer=memory)


Complex System Prompt

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful bot named Fred."),
    ("placeholder", "{messages}"),
    ("user", "Remember, always be polite!"),
])
# ¿Por qué no poner el recordatorio antes del placeholder?
# Si lo pusieras antes, el recordatorio podría quedar "enterrado" entre mensajes antiguos y perder relevancia. 
# Al ponerlo después, es lo último que ve el modelo antes de responder, 
# así que es más probable que lo tenga en cuenta en su respuesta.
graph = create_react_agent(llm, [calculator], prompt=prompt)
inputs = {"messages": [("user", "What's your name? Sum a random number to 4")]}
for s in graph.stream(inputs, stream_mode="values"):
    pprint(s)
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

{'messages': [HumanMessage(content="What's your name? Sum a random number to 4", additional_kwargs={}, response_metadata={}, id='2baac3a6-3801-4c6b-ae09-31479f91461e')]}
================================ Human Message =================================

What's your name? Sum a random number to 4
{'messages': [HumanMessage(content="What's your name? Sum a random number to 4", additional_kwargs={}, response_metadata={}, id='2baac3a6-3801-4c6b-ae09-31479f91461e'),
              AIMessage(content='My name is Fred.', additional_kwargs={'function_call': {'name': 'calculator', 'arguments': '{"a": 4.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--75d1e7dc-e47c-4a33-b252-d70130029a51-0', tool_calls=[{'name': 'calculator', 'args': {'a': 4.0}, 'id': '48a96468-e3e4-4ddc-a606-64ebd3fb16e2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 9, 'to

In [5]:
# If you want to run the agent, you can uncomment the following lines:
# https://langchain-ai.github.io/langgraph/agents/run_agents/#additional-resources

# response = langgraph_agent_executor.invoke({"messages": [{"role": "user", "content": "Make a sum the number is 4"}]})

In [4]:
config = {"configurable": {"thread_id": "test-thread"}}
# If you want to start a new conversation, all you have to do is change the thread_id used

query = "Make a sum the number is 4"

for step in langgraph_agent_executor.stream(
    {"messages": [("user", query)]},
    config,
    stream_mode="values",
):
    print(step["messages"][-1].pretty_print())
    print("---")

for step in langgraph_agent_executor.stream(
    {"messages": [("user", "De que estábamos hablando?")]},
    config,
    stream_mode="values",
):
    print(step["messages"][-1].pretty_print())
    print("---")

================================ Human Message =================================

Make a sum the number is 4
None
---
================================== Ai Message ==================================
Tool Calls:
  calculator (29cca44e-61c4-4451-94eb-fbcda373582c)
 Call ID: 29cca44e-61c4-4451-94eb-fbcda373582c
  Args:
    a: 4.0
None
---
================================= Tool Message =================================
Name: calculator

7
None
---
================================== Ai Message ==================================

La suma del número aleatorio y 4 es 7.
None
---
================================ Human Message =================================

De que estábamos hablando?
None
---
================================== Ai Message ==================================

Estábamos hablando de usar una calculadora para sumar un número aleatorio a 4. La respuesta fue 7.
None
---
